In [74]:
import pandas as pd
import numpy as np
from scipy import stats

In [75]:
# df = pd.read_csv('../data/output_w_label.csv')
df = pd.read_csv('../data/dataset1.csv')
# df = df.drop(columns='Unnamed: 0')

In [76]:
df.head()

,ID,POS,SEQ,PreTime,PreSD,PreMean,InTime,InSD,InMean,PostTime,PostSD,PostMean
0,ENST00000000233,244,AAGACCA,0.00465,2.16,127.0,0.00640,3.90,127.0,0.00797,8.75,83.7
1,ENST00000000233,244,AAGACCA,0.02690,4.43,106.0,0.01860,10.00,123.0,0.00863,6.20,80.0
2,ENST00000000233,244,AAGACCA,0.00432,3.10,108.0,0.01200,8.26,125.0,0.01590,2.89,78.7
3,ENST00000000233,244,AAGACCA,0.00996,4.52,123.0,0.01750,8.51,128.0,0.00498,2.63,80.0
4,ENST00000000233,244,AAGACCA,0.00764,2.81,124.0,0.00772,4.22,126.0,0.00474,5.84,80.9


In [77]:
# SEQ one hot encoding

base_map = {
    "A": [1, 0, 0, 0],
    "C": [0, 1, 0, 0],
    "G": [0, 0, 1, 0],
    "T": [0, 0, 0, 1],
    "U": [0, 0, 0, 1]   # treat U same as T
}

# Expand into one-hot encoded row
def one_hot_flat(seq):
    return sum([base_map[b] for b in seq], [])  # flatten list of lists

In [78]:
one_hot_vectors = df["SEQ"].apply(one_hot_flat)

In [79]:
# Create DataFrame with 28 columns
one_hot_df = pd.DataFrame(
    one_hot_vectors.tolist(),
    columns=[f"pos{i+1}_{b}" for i in range(len(df["SEQ"][0])) for b in ["A","C","G","T"]]
)

# Merge with original
df_expanded = pd.concat([df, one_hot_df], axis=1)

In [80]:
# Logic:
# pos0: A, C, G, T (all)
# pos1: A, G, T
# pos2: A, G
# pos3: A
# pos4: C
# pos5: A, C, T
# pos6: A, C, G, T (all)

df_expanded = df_expanded.drop(columns=['pos1_C', 'pos2_C', 'pos2_T', 'pos3_C', 'pos3_G', 'pos3_T','pos4_A', 'pos4_G', 'pos4_T','pos5_G'], axis=1)

In [81]:
df_expanded.head()

,ID,POS,SEQ,PreTime,PreSD,PreMean,InTime,InSD,InMean,PostTime,...,pos5_C,pos5_T,pos6_A,pos6_C,pos6_G,pos6_T,pos7_A,pos7_C,pos7_G,pos7_T
0,ENST00000000233,244,AAGACCA,0.00465,2.16,127.0,0.00640,3.90,127.0,0.00797,...,1,0,0,1,0,0,1,0,0,0
1,ENST00000000233,244,AAGACCA,0.02690,4.43,106.0,0.01860,10.00,123.0,0.00863,...,1,0,0,1,0,0,1,0,0,0
2,ENST00000000233,244,AAGACCA,0.00432,3.10,108.0,0.01200,8.26,125.0,0.01590,...,1,0,0,1,0,0,1,0,0,0
3,ENST00000000233,244,AAGACCA,0.00996,4.52,123.0,0.01750,8.51,128.0,0.00498,...,1,0,0,1,0,0,1,0,0,0
4,ENST00000000233,244,AAGACCA,0.00764,2.81,124.0,0.00772,4.22,126.0,0.00474,...,1,0,0,1,0,0,1,0,0,0


In [93]:
df_expanded.columns

Index(['ID', 'POS', 'SEQ', 'PreTime', 'PreSD', 'PreMean', 'InTime', 'InSD',
       'InMean', 'PostTime', 'PostSD', 'PostMean', 'pos1_A', 'pos1_G',
       'pos1_T', 'pos2_A', 'pos2_G', 'pos3_A', 'pos4_C', 'pos5_A', 'pos5_C',
       'pos5_T', 'pos6_A', 'pos6_C', 'pos6_G', 'pos6_T', 'pos7_A', 'pos7_C',
       'pos7_G', 'pos7_T'],
      dtype='object')

In [94]:
if "label" not in df_expanded.columns:
  labels = pd.read_csv("../data/data.info.labelled")
  labels = labels.rename(columns={'transcript_id': 'ID', 'transcript_position': 'POS'})
  df_expanded = pd.merge(df_expanded, labels, on =['ID', 'POS'], how="left")

           gene_id               ID  POS  label
0  ENSG00000004059  ENST00000000233  244      0
1  ENSG00000004059  ENST00000000233  261      0
2  ENSG00000004059  ENST00000000233  316      0
3  ENSG00000004059  ENST00000000233  332      0
4  ENSG00000004059  ENST00000000233  368      0


In [95]:
df_expanded.head()

,ID,POS,SEQ,PreTime,PreSD,PreMean,InTime,InSD,InMean,PostTime,...,pos6_A,pos6_C,pos6_G,pos6_T,pos7_A,pos7_C,pos7_G,pos7_T,gene_id,label
0,ENST00000000233,244,AAGACCA,0.00465,2.16,127.0,0.00640,3.90,127.0,0.00797,...,0,1,0,0,1,0,0,0,ENSG00000004059,0.0
1,ENST00000000233,244,AAGACCA,0.02690,4.43,106.0,0.01860,10.00,123.0,0.00863,...,0,1,0,0,1,0,0,0,ENSG00000004059,0.0
2,ENST00000000233,244,AAGACCA,0.00432,3.10,108.0,0.01200,8.26,125.0,0.01590,...,0,1,0,0,1,0,0,0,ENSG00000004059,0.0
3,ENST00000000233,244,AAGACCA,0.00996,4.52,123.0,0.01750,8.51,128.0,0.00498,...,0,1,0,0,1,0,0,0,ENSG00000004059,0.0
4,ENST00000000233,244,AAGACCA,0.00764,2.81,124.0,0.00772,4.22,126.0,0.00474,...,0,1,0,0,1,0,0,0,ENSG00000004059,0.0


In [99]:
df_expanded[df_expanded['gene_id'].isna()]

,ID,POS,SEQ,PreTime,PreSD,PreMean,InTime,InSD,InMean,PostTime,...,pos6_A,pos6_C,pos6_G,pos6_T,pos7_A,pos7_C,pos7_G,pos7_T,gene_id,label
22431,ENST00000023939,99,CTGACTC,0.00531,2.630,113.0,0.00232,3.87,128.0,0.01360,...,0,0,0,1,0,1,0,0,NaN,NaN
22432,ENST00000023939,99,CTGACTC,0.00888,3.040,108.0,0.00232,3.18,114.0,0.00358,...,0,0,0,1,0,1,0,0,NaN,NaN
22433,ENST00000023939,99,CTGACTC,0.00666,3.070,112.0,0.00232,3.88,127.0,0.01260,...,0,0,0,1,0,1,0,0,NaN,NaN
22434,ENST00000023939,99,CTGACTC,0.01030,3.270,113.0,0.00449,4.85,119.0,0.00398,...,0,0,0,1,0,1,0,0,NaN,NaN
22435,ENST00000023939,99,CTGACTC,0.00465,2.590,111.0,0.00498,10.40,116.0,0.01020,...,0,0,0,1,0,1,0,0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7903088,ENST00000641756,2102,ATAACCG,0.01100,2.720,85.6,0.02650,3.16,92.3,0.00432,...,0,1,0,0,0,0,1,0,NaN,NaN
7903089,ENST00000641756,2102,ATAACCG,0.00432,1.160,85.5,0.00797,3.91,92.2,0.00531,...,0,1,0,0,0,0,1,0,NaN,NaN
7903090,ENST00000641756,2102,ATAACCG,0.00332,2.640,84.7,0.00830,3.11,91.9,0.00199,...,0,1,0,0,0,0,1,0,NaN,NaN
7903091,ENST00000641756,2102,ATAACCG,0.00598,1.920,87.1,0.00760,3.45,92.1,0.00631,...,0,1,0,0,0,0,1,0,NaN,NaN


In [82]:
# df_expanded_temp = df_expanded

In [83]:
# df_expanded = df_expanded_temp.head(500)

In [84]:
# Define your numeric columns (all one-hot encoded ones)
numeric_cols = ['PreTime', 'PreSD', 'PreMean', 'InTime', 'InSD', 'InMean', 'PostTime', 'PostSD', 'PostMean']

# Custom function for mode (since it returns a Series)
def mode_func(x):
    return stats.mode(x, keepdims=True)[0][0] if len(x) > 0 else np.nan

# Quantile functions with named attributes (for clean column names)
def q25(x):
    return x.quantile(0.25)
q25.__name__ = "p25"

def q75(x):
    return x.quantile(0.75)
q75.__name__ = "p75"

# Perform grouped aggregation
df_agg = (
    df_expanded
    .groupby(["gene_id", "ID", "POS", "label"], as_index=False)
    .agg({col: ['mean', 'median', mode_func, 'std', 'min', 'max', q25, q75] 
          for col in numeric_cols})
)

# Flatten the MultiIndex column names
new_columns = []
for col_tuple in df_agg.columns:
    # Keep base columns (grouping keys) unchanged
    if col_tuple[0] in ["gene_id", "ID", "POS", "label"]:
        new_columns.append(col_tuple[0])
    else:
        # Combine numeric column + stat name
        stat_name = col_tuple[1]
        # Clean up _mode_func → _mode
        if stat_name == "mode_func":
            stat_name = "mode"
        new_columns.append(f"{col_tuple[0]}_{stat_name}")

df_agg.columns = new_columns

KeyError: 'gene_id'

In [ ]:
df_agg.head()

,gene_id,ID,POS,label,PreTime_mean,PreTime_median,PreTime_mode,PreTime_std,PreTime_min,PreTime_max,...,PostSD_p25,PostSD_p75,PostMean_mean,PostMean_median,PostMean_mode,PostMean_std,PostMean_min,PostMean_max,PostMean_p25,PostMean_p75
0,ENSG00000000003,ENST00000373020,512,0,0.007247,0.00599,0.00266,0.004404,0.00266,0.0169,...,1.3900,1.8750,92.330000,92.55,92.3,2.258574,83.5,94.5,92.100,93.325
1,ENSG00000000003,ENST00000373020,689,0,0.009868,0.00764,0.00531,0.006946,0.00232,0.0279,...,1.7400,2.3700,89.385714,89.60,90.4,2.113122,84.3,93.5,88.500,90.600
2,ENSG00000000003,ENST00000373020,823,0,0.007456,0.00631,0.00598,0.003799,0.00299,0.0196,...,1.9800,2.6300,88.176190,88.20,84.7,2.420517,84.7,93.1,86.200,89.400
3,ENSG00000000003,ENST00000373020,830,0,0.007765,0.00641,0.00498,0.004869,0.00266,0.0226,...,1.5475,2.4925,80.110000,80.00,78.7,1.523120,77.9,83.8,78.775,81.150
4,ENSG00000000003,ENST00000373020,849,0,0.006785,0.00598,0.00398,0.003115,0.00332,0.0181,...,1.5800,2.1000,84.657143,84.40,82.9,2.048798,82.3,90.7,83.100,85.500


In [ ]:
print(df_agg.columns.tolist())

['gene_id', 'ID', 'POS', 'label', 'PreTime_mean', 'PreTime_median', 'PreTime_mode', 'PreTime_std', 'PreTime_min', 'PreTime_max', 'PreTime_p25', 'PreTime_p75', 'PreSD_mean', 'PreSD_median', 'PreSD_mode', 'PreSD_std', 'PreSD_min', 'PreSD_max', 'PreSD_p25', 'PreSD_p75', 'PreMean_mean', 'PreMean_median', 'PreMean_mode', 'PreMean_std', 'PreMean_min', 'PreMean_max', 'PreMean_p25', 'PreMean_p75', 'InTime_mean', 'InTime_median', 'InTime_mode', 'InTime_std', 'InTime_min', 'InTime_max', 'InTime_p25', 'InTime_p75', 'InSD_mean', 'InSD_median', 'InSD_mode', 'InSD_std', 'InSD_min', 'InSD_max', 'InSD_p25', 'InSD_p75', 'InMean_mean', 'InMean_median', 'InMean_mode', 'InMean_std', 'InMean_min', 'InMean_max', 'InMean_p25', 'InMean_p75', 'PostTime_mean', 'PostTime_median', 'PostTime_mode', 'PostTime_std', 'PostTime_min', 'PostTime_max', 'PostTime_p25', 'PostTime_p75', 'PostSD_mean', 'PostSD_median', 'PostSD_mode', 'PostSD_std', 'PostSD_min', 'PostSD_max', 'PostSD_p25', 'PostSD_p75', 'PostMean_mean', 'Post

In [ ]:
len(df_agg)

121838

In [ ]:
df_agg.to_csv('../data/output_site_level_dataset1.csv')